In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, LeakyReLU, Reshape, SimpleRNN
from tensorflow.keras.utils import to_categorical
import scipy.io
import numpy as np

# Load SVHN dataset in Format 2
def load_svhn_format_2(path):
    data = scipy.io.loadmat(path)
    images = np.array(data['X'])
    images = np.moveaxis(images, -1, 0)
    labels = data['y']
    labels[labels == 10] = 0  # replace label 10 with 0
    return images, labels

# Normalize image pixel values
def normalize_images(images):
    return images / 255.0

# Load and preprocess data
train_data, train_labels = load_svhn_format_2('/Users/shantambhuraria/Desktop/data_605/SkyViewSVHN/train_32x32.mat')
test_data, test_labels = load_svhn_format_2('/Users/shantambhuraria/Desktop/data_605/SkyViewSVHN/test_32x32.mat')
train_data = normalize_images(train_data)
test_data = normalize_images(test_data)
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)



### CNN Model

In [10]:
# Build CNN model
model1 = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model Summary
model1.summary()

# Train the model
cnn_model = model1.fit(train_data, train_labels, epochs=10, validation_split=0.2)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 15, 15, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 6, 6, 64)          0         
 g2D)                                                            
                                                                 
 flatten_3 (Flatten)         (None, 2304)              0         
                                                                 
 dense_5 (Dense)             (None, 128)              

In [13]:
# Evaluate the model
test_loss, test_acc = model1.evaluate(test_data, test_labels)
print('Test accuracy:', test_acc)

814/814 [==============================] - 2s 2ms/step - loss: 0.4331 - accuracy: 0.8745
Test accuracy: 0.8745006322860718


### GAN Model

In [14]:
# GAN Discriminator as Classifier
def build_gan_discriminator(input_shape):
    model = Sequential()
    model.add(Conv2D(64, (3, 3), strides=(2, 2), padding='same', input_shape=input_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Conv2D(64, (3, 3), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    return model
gan_model = build_gan_discriminator((32, 32, 3))
gan_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [15]:
gan_history = gan_model.fit(train_data, np.ones(len(train_data)), epochs=10, validation_split=0.2)

Epoch 1/10
1832/1832 [==============================] - 11s 6ms/step - loss: 0.0011 - accuracy: 0.9998 - val_loss: 1.0247e-07 - val_accuracy: 1.0000
Epoch 2/10
1832/1832 [==============================] - 10s 6ms/step - loss: 9.0798e-08 - accuracy: 1.0000 - val_loss: 6.6753e-08 - val_accuracy: 1.0000
Epoch 3/10
1832/1832 [==============================] - 10s 6ms/step - loss: 4.9132e-08 - accuracy: 1.0000 - val_loss: 3.6858e-08 - val_accuracy: 1.0000
Epoch 4/10
1832/1832 [==============================] - 10s 6ms/step - loss: 2.5728e-08 - accuracy: 1.0000 - val_loss: 1.6018e-08 - val_accuracy: 1.0000
Epoch 5/10
1832/1832 [==============================] - 10s 6ms/step - loss: 1.1374e-08 - accuracy: 1.0000 - val_loss: 6.2528e-09 - val_accuracy: 1.0000
Epoch 6/10
1832/1832 [==============================] - 10s 6ms/step - loss: 4.4447e-09 - accuracy: 1.0000 - val_loss: 2.2870e-09 - val_accuracy: 1.0000
Epoch 7/10
1832/1832 [==============================] - 10s 6ms/step - loss: 1.6154e-0

In [16]:
gan_results = gan_model.evaluate(test_data, np.ones(len(test_data)))
print("GAN Accuracy:", gan_results[1])

814/814 [==============================] - 1s 1ms/step - loss: 5.8207e-10 - accuracy: 1.0000
GAN Accuracy: 1.0


###  RNN Model


In [21]:
# RNN Model
def build_rnn_model(input_shape):
    model = Sequential()
    model.add(Reshape((-1, input_shape[2]), input_shape=input_shape))  # Reshape for RNN
    model.add(SimpleRNN(128, return_sequences=True))
    model.add(SimpleRNN(128))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    return model

rnn_model = build_rnn_model((32, 32, 3))
rnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])



In [22]:
# Train the RNN model
rnn_history = rnn_model.fit(train_data, train_labels, epochs=10, validation_split=0.2)


Epoch 1/10
1832/1832 [==============================] - 554s 302ms/step - loss: 2.2444 - accuracy: 0.1871 - val_loss: 2.2377 - val_accuracy: 0.1875
Epoch 2/10
1832/1832 [==============================] - 555s 303ms/step - loss: 2.2381 - accuracy: 0.1895 - val_loss: 2.2378 - val_accuracy: 0.1875
Epoch 3/10
 994/1832 [===============>..............] - ETA: 3:59 - loss: 2.2392 - accuracy: 0.1891

KeyboardInterrupt: 

In [ ]:
# Evaluate the RNN model
rnn_results = rnn_model.evaluate(test_data, test_labels)
print("RNN Accuracy:", rnn_results[1])

### # Hybrid Model (CNN + RNN)

In [18]:
def build_hybrid_model(input_shape):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Reshape((-1, 64)))  # Prepare for RNN
    model.add(SimpleRNN(128, return_sequences=True))
    model.add(SimpleRNN(128))
    model.add(Dense(10, activation='softmax'))
    return model

hybrid_model = build_hybrid_model((32, 32, 3))
hybrid_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [19]:
hybrid_history = hybrid_model.fit(train_data, train_labels, epochs=10, validation_split=0.2)

Epoch 1/10
1832/1832 [==============================] - 35s 19ms/step - loss: 1.0588 - accuracy: 0.6542 - val_loss: 0.7655 - val_accuracy: 0.7572
Epoch 2/10
1832/1832 [==============================] - 35s 19ms/step - loss: 0.6746 - accuracy: 0.7892 - val_loss: 0.6831 - val_accuracy: 0.7865
Epoch 3/10
1832/1832 [==============================] - 35s 19ms/step - loss: 0.6167 - accuracy: 0.8086 - val_loss: 0.5989 - val_accuracy: 0.8164
Epoch 4/10
1832/1832 [==============================] - 35s 19ms/step - loss: 0.5825 - accuracy: 0.8206 - val_loss: 0.7027 - val_accuracy: 0.7802
Epoch 5/10
1832/1832 [==============================] - 35s 19ms/step - loss: 0.5794 - accuracy: 0.8229 - val_loss: 0.6132 - val_accuracy: 0.8152
Epoch 6/10
1832/1832 [==============================] - 35s 19ms/step - loss: 0.5691 - accuracy: 0.8260 - val_loss: 0.5399 - val_accuracy: 0.8387
Epoch 7/10
1832/1832 [==============================] - 36s 20ms/step - loss: 0.5680 - accuracy: 0.8276 - val_loss: 0.5959 -

In [20]:
hybrid_results = hybrid_model.evaluate(test_data, test_labels)
print("Hybrid Model Accuracy:", hybrid_results[1])

814/814 [==============================] - 4s 5ms/step - loss: 0.8498 - accuracy: 0.7406
Hybrid Model Accuracy: 0.7405501008033752
